## READme:

Jupyter Notebook & train data for recreating the normative charts from [Dimitrova et al., (2020)](https://www.biorxiv.org/content/10.1101/2020.08.05.228700v1.abstract). This notebook will allow the user to:

- run Gaussian Process Regression (GPR) on the normative volumetric data provided in ./data/
- fit the model to new test dataset (e.g. mydata.csv & mycov.csv) provided by the user
- calculate Z-scores for the new dataset based on the normative charts
- plot the new dataset overlaid onto the normative charts

The data provided here include sMRI scans from 274 term-born infants from the [developing Human Connectome Project](http://www.developingconnectome.org/), preprocessed with the official [dHCP structural pipeline](https://github.com/BioMedIA/dhcp-structural-pipeline) [(Makropoulos et al., 2018)](https://pubmed.ncbi.nlm.nih.gov/29409960/). For more information regarding the sample check out [Dimitrova et al., (2020)](https://www.biorxiv.org/content/10.1101/2020.08.05.228700v1.abstract).

Please note this is an initial commit and the notebook requires the holdout dataset/covs to be provided in a specific format. Please open and run the notebook from the current working directory. 

--------------
### Input data provided by the user:

- `mydata.csv` - a csv file with no header, containing the volumes in **mm3**, where regions are arranged as follow: 
           
           
     - Regions:           
            - CSF 
            - cortical Grey Matter (cGM)
            - White Matter (WM)
            - Ventricles (lateral ventricles + cavum)
            - Cerebellum
            - Brainstem
            - Caudate Right
            - Caudate Left
            - Lentiform Right (Pallidum & Putamen)
            - Lentiform Left (Pallidum & Putamen)
            - Thalamus Left
            - Thalamus Right
            - Total Tissue Volume (TTV) - All brain GM & WM tissue
            - Total Brain Volume (TBV) - All brain GM & WM tissue & ventricles
            - Intracranila Volume (ICV) - All brain GM & WM tissue & ventricles & CSF

- `mycov.csv` - a csv file with no header, where the covariates are arranges as follow: 

            - PMA at scan (in weeks) e.g. for an infants scanned at 40 1/7 this will be 40.14
            - sex 1 = male; 2 = female

\**each row represents a subject*\*


---------------------
### Output data:

Creates two directories: `./zscores` and `./figures`

- ./zscores/`my_holdout_Zscores.csv` - Z-scores from data provided in mydata.csv

- ./figures/`mydata_female_{}.png` - plots showing mydata overlaid onto the normative charts
          

------------------
### Dependencies:

- python => 3.7
- numpy
- pandas
- [GPy](https://github.com/SheffieldML/GPy)
- matplotlib


-------------
### Load packges:

In [ ]:
import pandas as pd             # pd.__version__         '0.25.1'
import numpy as np              # np.__version__         '1.17.2'
import matplotlib.pyplot as plt # matplotlib.__version__ '3.1.1'
import GPy                      # GPy.__version__        '1.9.6'
from datetime import datetime
import os

class style:
   p = '\033[95m'
   b = '\033[94m'
   bold = '\033[1m'
   end = '\033[0m'

-----------------
### Load Normative dHCP data:

1. Volumetric data: 
    - `term-train-0dot5mm.csv` - original dHCP 0.5mm resolution (274 infants)
    - `term-train-1mm.csv` - downsampled 1mm resolution (263 infants)
 
 *(change in cell)*
    

2. Covariates - `csv` file with data in order - PMA, sex, where:
 
    - `PMA at scan`: postmenstrual age at scan as floats 
    - `sex`: where 1 = male and 2 = female
    
Use `term_train_cov_0dot5mm.csv` or `term_train_cov_1mm.csv` depending on which resolution is chosen.

In [ ]:
data_dir = os.path.join(os.getcwd(), 'data')

vol_n = pd.read_csv('{}/term_train_0dot5mm.csv'.format(data_dir))
regions = list(vol_n.columns)
vol_n = vol_n.values
print()
print(style.bold +'Loaded:'+ style.end, 'normative data for {} dHCP infants'.format(vol_n.shape[0]))

cov = pd.read_csv('{}/term_train_cov_0dot5mm.csv'.format(data_dir), header = None).values
print(style.bold +'Loaded:'+ style.end, 'covariates (PMA & sex) for {} dHCP infants'.format(cov.shape[0]))

growth_cov = pd.read_csv('{}/growth_cov.csv'.format(data_dir), header = None).values
print(style.bold +'Loaded:'+ style.end, 'growth data between {} and {} weeks PMA for visualisation'.format(int(growth_cov[0,0:1]), 
                                                               int(growth_cov[(growth_cov.shape[0]-1),0:1])))

----------------
### Loading hold-out data: [insert your data here]

Here is where your data should come in. This could be achieved by:

1. copying `mydata.csv` (extracted volumetric data) and `mycov.csv` (predictors - PMA & sex) OR

2. directing python to the directory, where your data live

*Note: Ensure the volumes are arranged as shown above and that the covs are included as directed.*

In [ ]:
vol_hold_n = pd.read_csv('{}/mydata.csv'.format(data_dir)).values
print()
print(style.bold +'Loaded:'+ style.end, 'hold-out data')

holdout_cov = pd.read_csv('{}/mycov.csv'.format(data_dir)).values
print(style.bold +'Loaded:'+ style.end, 'covariates for hold-out data')
print()
print(style.b +'Now run the nex cell for a basic sanity check ensuring \n'
      'the training data and mydata have the same dimensions' + style.end)

In [ ]:
print()
print(style.bold +'Quick sanity check:'+ style.end)
print()
if vol_hold_n.shape[0] == holdout_cov.shape[0]:
    print(style.bold +'CHECK:'+ style.end ,'mydata and mycov have the same numbers of rows (infants)')
else:
    print(style.bold + style.p +'CHECK:'+ style.end ,'mydata and mycov have NOT the same number of rows (infants)')
    
if vol_n.shape[1] == vol_hold_n.shape[1]:
    print(style.bold +'CHECK:'+ style.end ,'mydata has the same No. of volumes as train data')
else:
    print(style.bold + style.p + 'CHECK:'+ style.end ,'mydata does NOT have the same No of volumes as train data')

if np.array_equal(np.unique(cov[:,1:2]), np.unique(holdout_cov[:,1:2])):
    print(style.bold +'CHECK:'+ style.end ,'mydata has the same labels (1,2) for male/female. Still ensure 1 = male/2 = female.')
else:
    print(style.bold + style.p +'CHECK:'+ style.end ,'mydata does NOT have the same labels (1,2) for male/female')   

print()
print(style.bold + style.b +'Quick Summary:'+ style.end, style.b +'{} infants scanned between {:0.2f} and {:0.2f} weeks PMA '
      '(mean: {:0.2f}); {:0.2f}% female'.format(vol_hold_n.shape[0], 
                                        np.min(holdout_cov[:,0:1]), 
                                        np.max(holdout_cov[:,0:1]),
                                        np.mean(holdout_cov[:,0:1]),
                                        (np.count_nonzero(holdout_cov[:,1:2] == 2))/holdout_cov.shape[0])+ style.end)

-------------------
### Have a look at the distribution of the covariates:

plots showing the PMA distribution and sex proportion for mydata and term data.


In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (8, 3))

# histogram: PMA at scan:
bins = np.unique(np.round(np.vstack((cov[:,0:1], holdout_cov[:,0:1])))).shape[0]
ax1.hist((cov[:,0:1],holdout_cov[:,0:1]), label = ['term', 'mydata'], bins = bins)
ax1.set_title('PMA at scan')
ax1.set_ylabel('No. infants')
ax1.set_xlabel('PMA at scan (weeks)')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.legend(loc = 2 , bbox_to_anchor = (0.01,1.01), borderaxespad = 0.)

# proportion bar plot: sex proportion:
base = [np.count_nonzero(cov[:,1:2] == 1)/cov.shape[0], 
               np.count_nonzero(holdout_cov[:,1:2] == 1)/holdout_cov.shape[0]]
ax2.bar(['term','mydata'], base, label = 'male', width = 0.5, color = 'darkblue')
ax2.bar(['term','mydata'], [np.count_nonzero(cov[:,1:2] == 2)/cov.shape[0], 
               np.count_nonzero(holdout_cov[:,1:2] == 2)/holdout_cov.shape[0]],
               bottom = base, label = 'female', width = 0.5, color = "teal")
ax2.set_title('Proportion female/male')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.legend(bbox_to_anchor = (1,1),borderaxespad = 0.)

-----------------------

### Estimating relative volumes 
 
1. Note: This converts the volume data from mm^3 to cm^3 for nicer visualisation.
 
 
2. Calculates `relative` volumes as:

 - CSF from ICV;
 - ventricles from TBV;
 - cGM, WM, subcortical from TTV.

In [ ]:
# from mm3 to cm3:
vol_n = vol_n / 1000
vol_hold_n = vol_hold_n / 1000 # comment out if your data are already in cm^3

ttv = vol_n[:,12:13]
tbv = vol_n[:,13:14]
icv = vol_n[:,14:15]
vol = np.hstack(((vol_n [:,0:1]/icv),(vol_n [:,1:3]/ttv),(vol_n [:,3:4]/tbv), (vol_n [:,4:12]/ttv), ttv, tbv, icv))

ttv = vol_hold_n[:,12:13]
tbv = vol_hold_n[:,13:14]
icv = vol_hold_n[:,14:15]
vol_hold = np.hstack(((vol_hold_n [:,0:1]/icv),(vol_hold_n [:,1:3]/ttv),(vol_hold_n [:,3:4]/tbv), 
                 (vol_hold_n [:,4:12]/ttv), ttv, tbv, icv))

-------------

##### Demean data:

In [ ]:
# demean covariates:
cov_demean = (cov - (np.mean(cov, axis = 0))) / np.std(cov, axis = 0)
holdout_cov_demean = (holdout_cov - (np.mean(cov, axis = 0)))/ np.std(cov, axis = 0)
growth_cov_demean = (growth_cov - (np.mean(cov, axis = 0)))/ np.std(cov, axis = 0)

# demean data:
vol_demean = (vol - (np.mean(vol, axis = 0))) / np.std(vol, axis = 0)
vol_demean_hold = (vol_hold - (np.mean(vol, axis = 0))) / np.std(vol, axis = 0)

print()
print('Data successfully demeaned.')

----------------
##### Prep data for fitting

In [ ]:
rois = vol.shape[1]
n_subj = cov.shape[0]
n_hold = holdout_cov.shape[0]
n_growth = growth_cov.shape[0]

# create an empty matrix
pred_term = np.zeros((n_subj,rois), dtype = 'float32')
var_term = np.zeros((n_subj,rois), dtype = 'float32')

pred_hold = np.zeros((n_hold,rois), dtype = 'float32')
var_hold = np.zeros ((n_hold,rois), dtype = 'float32')

pred_growth = np.zeros((n_growth,rois), dtype = 'float32')
var_growth = np.zeros ((n_growth,rois), dtype = 'float32')

test_indices_hold = np.hstack([holdout_cov_demean, np.zeros_like(holdout_cov_demean[:,0:1])])
test_indices_growth = np.hstack([growth_cov_demean, np.zeros_like(growth_cov_demean[:,0:1])])

model_list = []

-----------------
### Gaussian Process Regression:


In [ ]:
current_time = datetime.now().strftime("%H:%M:%S")
print()
print("Start of fitting @", current_time)

for roi in range(0,rois):    
    print('{}: Fitting {}'.format((roi+1), regions[roi]))

    # initial fit on the whole data to predict hold-out & growth:   
    base_vol = vol_demean[:,roi:(roi+1)]    
    train_image = base_vol 
    train_age = cov_demean
    
    # set kernel:
    k1 = GPy.kern.RBF(2, active_dims = (0,1), lengthscale=2)
    k2 = GPy.kern.White(2, active_dims = (0,1))
    k3 = GPy.kern.Linear(2, active_dims = (0,1))
    k_add = k1 + k2 + k3
    
    # fit GPR:   
    m = GPy.models.SparseGPRegression(train_age, train_image, kernel = k_add)    
    m.optimize('bfgs', max_iters=100)    
    model_list.append(m.param_array) 
    
    # predicts holdout data based on whole sample:
    pred_hold[:,roi:(roi + 1)], var_hold[:,roi:(roi + 1)] = m.predict(test_indices_hold) 
    pred_growth[:,roi:(roi + 1)], var_growth[:,roi:(roi + 1)] = m.predict(test_indices_growth)
    
    # creates an empty output vector to fill in with prediciton from 5CV:
    pred_5cv = np.zeros((n_subj,1), dtype = 'float32')
    var_5cv = np.zeros((n_subj,1), dtype = 'float32')
    
    # actual 5CV loop for if you want comparisons to the term Z-scores:
    for i in range (0,5):
        selection = np.arange(i, n_subj, 5)
        
        # split into train and test:
        test_age = cov_demean[selection,:] 
        test_indices = np.hstack([test_age, np.zeros_like(test_age[:,0:1])])     
        train_age = np.delete(cov_demean, [selection], axis = 0)
        
        test_vol = base_vol[selection,:]
        train_vol = np.delete(base_vol, selection, axis = 0)
        train_image = train_vol        
        target_indices = np.hstack(selection)
        
        # running the model:
        k1 = GPy.kern.RBF(2,active_dims = (0,1), lengthscale = 2)
        k2 = GPy.kern.White(2,active_dims = (0,1))
        k3 = GPy.kern.Linear(2,active_dims = (0,1))
        k_add = k1 + k2 + k3     
        
        # fit:
        m = GPy.models.SparseGPRegression(train_age, train_image, kernel= k_add)
        m.optimize('bfgs', max_iters=100)
        
        # predict under 5CV:
        pred_5cv[target_indices], var_5cv[target_indices] = m.predict(test_indices)
    
    # write in after 5 CV:
    pred_term[:, roi:(roi + 1)] = pred_5cv
    var_term[:, roi:(roi + 1)] = var_5cv
    
current_time = datetime.now().strftime("%H:%M:%S")
print("End of fitting @", current_time)

------------------
### Get Z-scores for hold-out data:

- creates a directory `./zscores`
- saves the Z-scores for mydata in `./zscores/my_holdout_Zscores.csv`


Note: if ran multple times, don't forget the file `./zscores/my_holdout_Zscores.csv` will be overwritten.

In [ ]:
# If it does not exist, creates a directory  ./zscores/:
zscores_dir = os.path.join(os.getcwd(), 'zscores')
print()
if os.path.exists(zscores_dir):
    print('{} directory exists'.format(zscores_dir))
else:
    os.mkdir(zscores_dir)
    print ('creating directory {}'.format(zscores_dir))
print()

# calculate Z-scores:
holdout_Zscore = (vol_demean_hold - pred_hold) / np.sqrt(var_hold)

# save Z-scores:
pd.DataFrame(holdout_Zscore, columns = regions).to_csv('./zscores/my_holdout_Zscores.csv', sep = ',', index = False)

print('Z-scores for mydata are saved in:\n', style.b +'{}/my_holdout_Zscores.csv'.format(
    os.path.realpath(zscores_dir))+ style.b)  #os.path.join(os.getcwd(), 'zscores')))+ style.b)

---------------------
### Visualise hold-out data onto the normative charts:


######  Prep data:

In [ ]:
# growth data in cm3:
growth_age = growth_cov [0:71, 0:1]
# mean:
growth_male = (pred_growth [0:71,:] * np.std(vol, axis = 0)) + np.mean(vol, axis = 0) 
growth_female = (pred_growth [71:,:] * np.std(vol, axis = 0)) + np.mean(vol, axis = 0) 
# std:
growth_male_std = np.sqrt(var_growth [0:71,:]) * np.std(vol, axis = 0) 
growth_female_std = np.sqrt(var_growth [71:,:]) * np.std(vol, axis = 0) 

# Split hold-out data into male & female infants:
data = np.concatenate([vol_hold, holdout_cov], axis = 1)
data_male = data[data[:, 16] == 1, :]
data_female = data[data[:, 16] == 2, :]
# creates the directory to save the figures created in the next two cells:
plots_dir = os.path.join(os.getcwd(), 'figures')
print()
if os.path.exists(plots_dir):
    print('{} directory exists'.format(plots_dir))
else:
    os.mkdir(plots_dir)
    print ('creating directory {}'.format(plots_dir))


###### And now the plots:

- Plots are created separately for female and male infants. 
- To save the plots uncomment the last line in both cells.

In [ ]:
# Normative charts for female infants: 

for i in range (0,len(regions)):
    d = growth_female
    std = growth_female_std
    pma = np.squeeze(growth_age)
    
    l1 = np.squeeze(d[:,i] + std[:,i])
    l2 = np.squeeze(d[:,i] - std[:,i])
    l3 = np.squeeze(d[:,i] + (2 * std[:,i]))
    l4 = np.squeeze(d[:,i] - (2 * std[:,i]))
    l5 = np.squeeze(d[:,i] + (3 * std[:,i]))
    l6 = np.squeeze(d[:,i] - (3 * std[:,i]))
    
    fig, ax = plt.subplots(1, figsize = (4.5,3))
    ax.plot(pma, d[:,i], lw = 3, label ='female mean', color = 'black') 
    ax.fill_between(pma, l1, l2, facecolor = 'rebeccapurple', alpha = 0.15)
    ax.fill_between(pma, l3, l4, facecolor = 'rebeccapurple', alpha = 0.12)
    ax.fill_between(pma, l5, l6, facecolor = 'rebeccapurple', alpha = 0.1)
    
    plt.scatter(data_female[:,15], data_female[:,i], s = 80, alpha = 0.6, label = 'my_data', color = "teal")
    ax.set_title('{}'.format(regions[i]))
    
    plt.legend(loc=1, prop={'size': 12}, bbox_to_anchor = (1.6,1),borderaxespad = 0.)

# uncomment below to save the plots:
    #plt.savefig('{}/mydata_female_{}.png'.format(os.path.realpath(plots_dir), regions[i]), bbox_inches='tight', dpi = 100)

In [ ]:
# Normative charts for male infants:

for i in range (0, len(regions)):
    d = growth_male
    std = growth_male_std
    pma = np.squeeze(growth_age)
    
    l1 = np.squeeze(d[:,i] + std[:,i])
    l2 = np.squeeze(d[:,i] - std[:,i])
    l3 = np.squeeze(d[:,i] + (2 * std[:,i]))
    l4 = np.squeeze(d[:,i] - (2 * std[:,i]))
    l5 = np.squeeze(d[:,i] + (3 * std[:,i]))
    l6 = np.squeeze(d[:,i] - (3 * std[:,i]))
    
    fig, ax = plt.subplots(1, figsize = (4.5,3))
    ax.plot(pma, d[:,i], lw = 3, label ='male mean', color = 'black') 
    ax.fill_between(pma, l1, l2, facecolor = 'steelblue', alpha = 0.15)
    ax.fill_between(pma, l3, l4, facecolor = 'steelblue', alpha = 0.12)
    ax.fill_between(pma, l5, l6, facecolor = 'steelblue', alpha = 0.1)
    
    plt.scatter(data_male[:,15], data_male[:,i], s = 80, alpha = 0.6, label = 'my_data', color = "royalblue")
    ax.set_title('{}'.format(regions[i]))
    
    plt.legend(loc=1, prop={'size': 12}, bbox_to_anchor = (1.6,1),borderaxespad = 0.)

# uncomment below to save the plots:
    #plt.savefig('{}/mydata_male_{}.png'.format(os.path.realpath(plots_dir), regions[i]), bbox_inches='tight', dpi = 100)